# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [4]:
# your code here
orders = pd.read_csv('Orders.csv')
orders.head(10)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
5,5,536365,22752,2010,12,3,8,set 7 babushka nesting boxes,2,2010-12-01 08:26:00,7.65,17850,United Kingdom,15.30
6,6,536365,21730,2010,12,3,8,glass star frosted t-light holder,6,2010-12-01 08:26:00,4.25,17850,United Kingdom,25.50
7,7,536366,22633,2010,12,3,8,hand warmer union jack,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,11.10
8,8,536366,22632,2010,12,3,8,hand warmer red polka dot,6,2010-12-01 08:28:00,1.85,17850,United Kingdom,11.10
9,9,536367,84879,2010,12,3,8,assorted colour bird ornament,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,54.08


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [ ]:
# your code here
#VIP Customers: aggregated expenses > the 95th percentile (aka. 0.95 quantile).
#Preferred Customers: 75th < aggregated expenses > 95th percentile

In [73]:
#Sub Problem 1: How to aggregate the amount_spent for unique customers?
c_avg = orders.groupby(['CustomerID', 'Country'], as_index=False).agg({'amount_spent':'sum'})
c_avg

,CustomerID,Country,amount_spent
0,12346,United Kingdom,77183.60
1,12347,Iceland,4310.00
2,12348,Finland,1797.24
3,12349,Italy,1757.55
4,12350,Norway,334.40
...,...,...,...
4342,18280,United Kingdom,180.60
4343,18281,United Kingdom,80.82
4344,18282,United Kingdom,178.05
4345,18283,United Kingdom,2094.88


In [74]:
#Sub Problem 2: How to select customers whose aggregated amount_spent is in a given quantile range?
#checking data
c_avg.describe()

,CustomerID,amount_spent
count,4347.000000,4347.000000
mean,15294.640212,2050.013320
std,1724.722441,8980.103578
min,12346.000000,0.000000
25%,13806.500000,307.430000
50%,15292.000000,671.960000
75%,16776.500000,1661.195000
max,18287.000000,280206.020000


In [85]:
#Hint:  c_avg["customer_type"] = c_avg[c_avg['amount_spent'] > c_avg['amount_spent'].quantile(.90)]
#quartiles I defined (could be q=10)
c_avg["customer_type"] = pd.qcut(c_avg['amount_spent'], q=4, labels = ('level 1', 'level 2', 'preferred', 'VIP'))
c_avg.sort_values(by='amount_spent', ascending=False)

,CustomerID,Country,amount_spent,customer_type
1698,14646,Netherlands,280206.02,VIP
4210,18102,United Kingdom,259657.30,VIP
3737,17450,United Kingdom,194550.79,VIP
3017,16446,United Kingdom,168472.50,VIP
1888,14911,EIRE,143825.06,VIP
...,...,...,...,...
4107,17956,United Kingdom,12.75,level 1
3023,16454,United Kingdom,6.90,level 1
1802,14792,United Kingdom,6.20,level 1
3226,16738,United Kingdom,3.75,level 1


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [158]:

#only VIP
filtered = c_avg[c_avg["customer_type"] == "VIP" ]
#sort and give me first : iloc for index, only numbers
count_ = pd.DataFrame(filtered.groupby(["Country"]).count()).sort_values(by=['customer_type'], ascending = False)
count_.iloc[0]


CustomerID       934
amount_spent     934
customer_type    934
Name: United Kingdom, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [159]:
#one or another
filtered = c_avg[ (c_avg["customer_type"] == "VIP") |  (c_avg["customer_type"] == "preferred" )]
#sort and give me first : iloc for index, only numbers
count_ = pd.DataFrame(filtered.groupby(["Country"]).count()).sort_values(by=['customer_type'], ascending = False)
count_.iloc[0]

CustomerID       1910
amount_spent     1910
customer_type    1910
Name: United Kingdom, dtype: int64